In [32]:
import os
import mlflow
from dotenv import load_dotenv
load_dotenv()

# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "bvv_43"
RUN_NAME = "log_model"

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 


In [ ]:
client = mlflow.MlflowClient(tracking_uri=tracking_uri, registry_uri=registry_uri)

In [33]:
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

#### Стата по эксперименту + метрики

In [34]:
experiment_runs = mlflow.search_runs(experiment_ids=[experiment_id],).sort_values(by="start_time", ascending=False) 

In [41]:
runs = experiment_runs[[
	"run_id", "start_time",	"metrics.err1", "metrics.err2", "metrics.logloss", "metrics.recall", "metrics.auc", "metrics.f1", "metrics.precision"]]

In [44]:
runs.head(30)

,run_id,start_time,metrics.err1,metrics.err2,metrics.logloss,metrics.recall,metrics.auc,metrics.f1,metrics.precision
0,56f312fdbf7844cba41574824f3e4566,2025-12-28 13:26:22.860000+00:00,0.2,0.35,7.208731,1.0,0.934066,0.777778,0.636364
1,8e0ff396803e419cb910bc5acbcc845e,2025-12-28 13:25:43.783000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62a3bdb7f0814789ba62eb05d04ae646,2025-12-27 19:36:10.978000+00:00,0.2,0.35,7.208731,1.0,0.934066,0.777778,0.636364
3,7abcdf3786d942d184f19f37a92d2ea9,2025-12-27 17:05:14.171000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a011534674f499792deb4bdcff1af34,2025-12-27 17:04:59.206000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4ae53a84931e43ec89d2a743799d531c,2025-12-27 11:07:40.114000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,a5b9156ad115423f837f4e0ea98bce8e,2025-12-27 11:07:21.017000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,37f2f597b05d42ce9df4410cfa53f046,2025-12-27 11:07:04.845000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7b4b5de13a0742d5a20725cf10ce53ed,2025-12-27 10:57:37.597000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,149f590609d14b2c88fc424a1f5ec8cc,2025-12-27 10:47:56.606000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# смотрим метрики не через таблицу, а сами значения метрик
# фиксируем интересующий запуск (run_id) и метрику
run_id = "56f312fdbf7844cba41574824f3e4566"
metric = "logloss"

print(client.get_metric_history(run_id, metric)) 

[<Metric: key='logloss', step=0, timestamp=1766928382884, value=7.20873067782343>]


In [47]:
#все метрики по конкретному запуску
client.get_run(run_id).data.metrics 

{'err1': 0.2,
 'err2': 0.35,
 'auc': 0.9340659340659341,
 'precision': 0.6363636363636364,
 'recall': 1.0,
 'f1': 0.7777777777777778,
 'logloss': 7.20873067782343}

#### Артефакты

все артефакты для выбранногоrun_id хранятся в «папке» под названием artifacts. Важно, что объекты в объектных хранилищах хранятся без иерархической структуры

внутри одного запуска у всех артефактов одинаковый базовый uri, который позволяет скачать их все одновременно. Вот как можно получить uri:

In [49]:
run = client.get_run(run_id)
artifact_uri = run.info.artifact_uri

print(f"Путь артефакта в S3: '{artifact_uri}'") 

Путь артефакта в S3: 's3://s3-student-mle-20251108-d3b615298f-freetrack/8/56f312fdbf7844cba41574824f3e4566/artifacts'


In [ ]:
# Чтобы скачать артефакты в path="./artifacts_local", необходимо выполнить код:
mlflow.artifacts.download_artifacts(artifact_uri, dst_path="./artifacts_local")

'/home/mle-user/mle_projects/mle-mlflow/artifacts_local/artifacts'

In [52]:
#Аналогичным образом можно достать и модель, так как файл с моделью — тоже артефакт
REGISTRY_MODEL_NAME = "churn_model_bvv43"
REGISTRY_MODEL_VERSION = "1"

model_uri = client.get_model_version_download_uri(
    REGISTRY_MODEL_NAME,
    REGISTRY_MODEL_VERSION,
)

#model = mlflow.catboost.load_model(model_uri)
model = mlflow.sklearn.load_model(model_uri)

print(f"Путь до модели {REGISTRY_MODEL_NAME} версии {REGISTRY_MODEL_VERSION} в S3: '{model_uri}'") 


Путь до модели churn_model_bvv43 версии 1 в S3: 's3://s3-student-mle-20251108-d3b615298f-freetrack/8/4ae53a84931e43ec89d2a743799d531c/artifacts/models'
